In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

import requests
import re
import time
import datetime

from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
from bs4 import BeautifulSoup

sns.set()

In [2]:
PATH = "C:/Users/ADMIN/OneDrive - Trường ĐH CNTT - University of Information Technology/Tài liệu/Tài liệu học tập/DS300/FinalProject/Dataset/"

# Crawl data from VietNamWork

In [3]:
# Viết hàm lấy url page chứa danh sách các công việc
def get_url_page_VNW(num_page):
  list_url_page = []
  for i in range(1,num_page+1):
    list_url_page.append('https://www.vietnamworks.com/viec-lam?page='+str(i))
  return list_url_page

In [4]:
list_url_page = get_url_page_VNW(500)

In [5]:
driver = webdriver.Chrome()

In [6]:
def get_url_job_VNW(url_page):
  # Khởi tạo danh sách url các công việc
  list_url_job = []
  # Lấy nội dung của page
  
  driver.get(url_page)

  last_height = driver.execute_script("return document.body.scrollHeight")
  while True:
      # Scroll to the bottom
      driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
      
      # Wait for new content to load (adjust time as needed)
      time.sleep(5)
      
      # Calculate new scroll height
      new_height = driver.execute_script("return document.body.scrollHeight")
      
      # Break the loop if no new content is loaded
      if new_height == last_height:
          break
      last_height = new_height

      
  url_jobs = driver.find_elements(By.CSS_SELECTOR, 
      "#vnwLayout__col > div > div:nth-child(1) > div.col-12.col-lg-9.no-padding-left.no-padding-responsive.search-result-listJob > div.sc-2d549b0c-0.hgDWoS > div > div.search_list.view_job_item > div > div > div.sc-euWMRQ.dgXSCK > div.sc-gpaZuh.jzkZXw > div > div > div > h2 > a"
  )
  for job in url_jobs:
    list_url_job.append(job.get_attribute('href'))
  
  return list_url_job

In [ ]:
list_url_job_VNW = []
for url_page in tqdm(list_url_page):
  try:
    list_url_job_VNW.extend(get_url_job_VNW(url_page))
  except:
    print(f'{url_page} -> Error')

In [ ]:
len(list_url_job_VNW)

6781

In [ ]:
list_url_job_VNW

['https://www.vietnamworks.com/bd-ux-researcher-service-design-1834430-jv?source=searchResults&searchType=2&placement=1834430&sortBy=date',
 'https://www.vietnamworks.com/senior-expert-economics-and-financial-markets-analysis--1825011-jv?source=searchResults&searchType=2&placement=1825011&sortBy=date',
 'https://www.vietnamworks.com/audit-senior-ha-noi-hcm-1834753-jv?source=searchResults&searchType=2&placement=1834753&sortBy=date',
 'https://www.vietnamworks.com/qa-engineer-ky-su-chat-luong-tieng-anh-thanh-thao--1840419-jv?source=searchResults&searchType=2&placement=1840419&sortBy=date',
 'https://www.vietnamworks.com/truong-nhom-kinh-doanh-1842810-jv?source=searchResults&searchType=2&placement=1842810&sortBy=date',
 'https://www.vietnamworks.com/truong-phong-kinh-doanh-kenh-le-chi-nhanh-mien-bac-1830609-jv?source=searchResults&searchType=2&placement=1830609&sortBy=date',
 'https://www.vietnamworks.com/retail-leasing-senior-assistant-manager-manager-1840638-jv?source=searchResults&sear

In [ ]:
df_url_job_TCV = pd.DataFrame({'URL Job': list_url_job_VNW})

In [29]:
df_url_job_TCV.to_csv(PATH+'/job_url_TCV.csv',index_label='ID')

In [3]:
data = pd.read_csv(PATH + '/job_url_TCV.csv')

In [3]:
def Crawl_job_data(url):
    content_job = requests.get(url).content
    soup_job = BeautifulSoup(content_job, 'html.parser')

    # Tiêu đề công việc
    job_title = soup_job.find('h1', {'class':'sc-df6f4dcb-0 bsKseP'}).text.strip()

    # Tên công ty
    company_name = soup_job.find('a', {'class': 'sc-df6f4dcb-0 dIdfPh sc-f0821106-0 gWSkfE'}).text.strip()

    # Địa chỉ công ty
    company_address = soup_job.select('div.sc-37577279-3.drWnZq > div > div:nth-child(1) > div:nth-child(2) > p')[0].text.strip()

    # Mô tả công việc
    description = soup_job.find_all('div', {'class': 'sc-4913d170-6 hlTVkb'})[0].find_all('p')
    job_description = '. '.join(
        p.get_text(strip=True).lstrip("•").strip() for p in description if p.get_text(strip=True))
    job_description = re.sub(r'\.+', '.', job_description)
    job_description = re.sub(r"-\s*", "", job_description)

    # Yêu cầu công việc
    requirements = soup_job.find_all('div', {'class': 'sc-4913d170-6 hlTVkb'})[1].find_all('p')
    job_requirement = '. '.join(
        p.get_text(strip=True).lstrip("•").strip() for p in requirements if p.get_text(strip=True)
    )

    job_requirement = re.sub(r'\.+', '.', job_requirement)
    job_requirement = re.sub(r"-\s*", "", job_requirement)

    # Phúc lợ
    welfares = soup_job.find_all('div', {'class': 'sc-c683181c-2 fGxLZh'})
    job_welfares = '. '.join(div.get_text(strip=True) for div in welfares) + '.'
    job_welfares: str = re.sub(r"-\s*", "", job_welfares)

    # Mức lương
    salary = soup_job.find_all('span', {'class': 'sc-df6f4dcb-0 iOaLcj'})[0].text.strip()

    # Ngày hết hạn
    expire_date = soup_job.find_all('span', {'class': 'sc-df6f4dcb-0 bgAmOO'})[0].text.strip()

    elements = soup_job.find_all('p', {'class': 'sc-df6f4dcb-0 ioTZSy'})
    employee_rank = elements[1].text.strip()  # Cấp bậc
    job_field = elements[2].text.strip() # Ngành nghề
    job_skill = elements[3].text.strip() # Kĩ năng
    Industry = elements[4].text.strip() # Lĩnh vực
    experiences = elements[6].text.strip() # Số năm kinh nghiệm
    job_address = elements[-1].text.strip() # Địa chỉ làm việc


    return [job_title, company_name, Industry, company_address, job_description, job_requirement, job_welfares, salary, expire_date, employee_rank, job_skill, job_field, experiences, job_address]

In [4]:
job_url, job_title, company_name, Industry, company_address, job_description, job_requirement = [], [], [], [], [], [], []
job_welfares, salary, expire_date, employee_rank, job_skill, job_field, experiences, job_address = [], [], [], [], [], [], [], []

In [ ]:
for url in tqdm(data["URL Job"]):
  try:
    job_data_TCV = Crawl_job_data(url)
    job_url.append(url)
    job_title.append(job_data_TCV[0])
    company_name.append(job_data_TCV[1])
    Industry.append(job_data_TCV[2])
    company_address.append(job_data_TCV[3])
    job_description.append(job_data_TCV[4])
    job_requirement.append(job_data_TCV[5])
    job_welfares.append(job_data_TCV[6])
    salary.append(job_data_TCV[7])
    expire_date.append(job_data_TCV[8])
    employee_rank.append(job_data_TCV[9])
    job_skill.append(job_data_TCV[10])
    job_field.append(job_data_TCV[11])
    experiences.append(job_data_TCV[12])
    job_address.append(job_data_TCV[13])
  except:
    print('Error')

  1%|          | 44/6781 [00:18<45:44,  2.45it/s]  

Error


  1%|          | 72/6781 [00:32<51:08,  2.19it/s]  

Error


  2%|▏         | 103/6781 [00:46<44:51,  2.48it/s] 

...

100%|█████████▉| 6763/6781 [51:51<00:13,  1.34it/s]

Error


100%|█████████▉| 6766/6781 [51:52<00:08,  1.70it/s]

Error


100%|██████████| 6781/6781 [52:00<00:00,  2.17it/s]


In [ ]:
Dataset_VNW = pd.DataFrame({'URL Job': job_url,
                            'Job Title': job_title,
                            'Company Name': company_name,
                            'Industry': Industry,
                            'Company Address': company_address,
                            'Job Description': job_description,
                            'Job Requirements': job_requirement,
                            'Benefits': job_welfares,
                            'Salary': salary,
                            'Expire date': expire_date,
                            'Employee Rank': employee_rank,
                            'Job Skill': job_skill,
                            'Job Field': job_field,
                            'Years of Experience': experiences,
                            'Job Address': job_address})

In [10]:
Dataset_VNW

,URL Job,Job Title,Name Company,Company Size,Company Address,City,Job Description,Job Requirements,Benefits,Salary,Expire date,Employee Rank,Job Skill,Job Field,Years of Experience,Job Address
0,https://www.vietnamworks.com/bd-ux-researcher-...,BD - UX Researcher (Service Design),Bosch Global Software Technologies Company Lim...,1.000-4.999 nhân viên,"Etown 2, 364 Cong Hoa street, ward 13, Tan Bin...",Hồ Chí Minh,Your Contribution to Something Big. Shape the ...,Diploma or Bachelor’s/Master’s degree in Psych...,13th-month salary bonus + attractive performan...,Thương lượng,Hết hạn trong 3 ngày,Nhân viên,"Communication, Critical Thinking, Service Desi...",Phần Mềm CNTT/Dịch vụ Phần mềm,Bất kỳ,"Etown 2, HCMC"
1,https://www.vietnamworks.com/senior-expert-eco...,"Senior Expert, Economics and Financial Markets...",Techcombank,5.000-9.999 nhân viên,"Head Office: 6 Quang Trung Street, Hoan Kiem D...",Hà Nội,Job Purpose. The job holder responsible for:. ...,"- Degree: University degree in Finance, Econom...",13th month salary + performance bonus. Advance...,"$ 1,000-2,500 /tháng",Hết hạn trong 7 ngày,Trưởng phòng,"Banking, Financial Analysis, Macro Analysis, F...",Ngân hàng,Tiếng Anh,"Số 6 Quang Trung, Hoàn Kiếm, HN"
2,https://www.vietnamworks.com/audit-senior-ha-n...,Audit Senior ( Ha Noi & HCM),"Crowe Vietnam Co., Ltd",100-499 nhân viên,"HAN Office: Room 1205, 12th Floor, IPH Tower, ...","Hà Nội, Hồ Chí Minh",1. The primary responsibility is to assist aud...,- University degree in Auditing/Accounting/ Fi...,Work for an international brand that frequentl...,Thương lượng,Hết hạn trong 3 ngày,Nhân viên,"Audit, Analytical Skills, Kiểm Toán Nội Bộ, En...",Kế toán/Kiểm toán,Tiếng Anh,"Hồ Chí Minh, Việt Nam"
3,https://www.vietnamworks.com/qa-engineer-ky-su...,"QA Engineer (Kỹ Sư Chất Lượng, Tiếng Anh Thành...",Công Ty TNHH LG Innotek Việt Nam Hải Phòng,1.000-4.999 nhân viên,"Lô I, 4, Khu công nghiệp Tràng Duệ, Xã Hồng Ph...",Hải Phòng,Process Quality Assurance. - The Process Quali...,- Well basis quality concept. - English commun...,Công ty cung cấp ký túc xá (nếu cần thiết). Xe...,"Tới $ 1,000 /tháng",Hết hạn trong 20 ngày,Nhân viên,"Process Quality Assurance, Excel, Test Plannin...",Điện/Điện tử,Bất kỳ,"Hai Phong, Vietnam"
4,https://www.vietnamworks.com/truong-nhom-kinh-...,Trưởng Nhóm Kinh Doanh,AIA Exchange HCM,Mr Đoàn Ngọc Hưng,"Vincom Center, 45 Lý Tự Trọng, Phường Bến Nghé...",Hồ Chí Minh,Lên kế hoạch tuyển dụng và phát triển đội ngũ ...,Độ tuổi: 27 - 40. Có kinh nghiệm ít nhất 1 năm...,Thu nhập: Ít nhất từ 25 - 40 triệu mỗi tháng. ...,25tr-40tr ₫/tháng,Hết hạn trong 28 ngày,Nhân viên,"Quản Trị Đội Ngũ, Kỹ năng bán hàng, Quản Lý Tà...",Bảo hiểm,Tiếng Việt,"Tầng 15, Vincom Đồng Khởi, 45A Lý Tự Trọng, Q1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6377,https://www.vietnamworks.com/internal-auditor-...,Internal Auditor- APAC region ( fluent in Engl...,Navigos Search,1.000-4.999 nhân viên,HCMC,"Hồ Chí Minh, Đồng Nai",Main Responsibilities. Plan and execute audits...,Minimum 4 years of experience. Ideally having ...,Professional and personal development through ...,"$ 1,500-2,000 /tháng",Hết hạn trong 7 giờ,Nhân viên,"Analytical Skills, Audit Management, English F...",Sản xuất,Bất kỳ,khu công nghiệp tam phước
6378,https://www.vietnamworks.com/urgent-production...,[Urgent] Production Manager working in Củ Chi,Navigos Search,1.000-4.999 nhân viên,HCMC,Hồ Chí Minh,Assures production shift target are met or exc...,Bachelor's degree or higher in engineering or ...,Fixed 13th salary.. Personal healthcare insura...,Thương lượng,Hết hạn trong 7 giờ,Nhân viên,"English Fluency, Leadership Skills, Manufactur...",Sản xuất,Tiếng Anh,"Củ Chi, Thành phố Hồ Chí Minh, Vietnam"
6379,https://www.vietnamworks.com/urgent-production...,[Urgent] Production Supervisor working in Củ Chi,Navigos Search,1.000-4.999 nhân viên,HCMC,Hồ Chí Minh,Plan and manage equipment and manpower to meet...,Education: A Bachelor’s degree or equivalent i

In [11]:
Dataset_VNW.to_csv(PATH+'/Dataset_VNW', index = False)

In [40]:
data = pd.read_csv(PATH+'/Dataset_VNW')

In [48]:
data['Years of Experience'].value_counts()

Years of Experience
Bất kỳ              4370
Tiếng Anh           1552
Tiếng Việt           372
Tiếng Trung Quốc      47
Tiếng Nhật            29
Tiếng Hàn Quốc        11
Tiếng Pháp             1
Name: count, dtype: int64

# Crawl job data from Jobsgo


In [2]:
def get_url_page_Jobsgo(num_page):
  list_url_page = []
  for i in range(1,num_page+1):
    list_url_page.append(f'https://jobsgo.vn/viec-lam.html?utm_source=job_list&utm_medium=&utm_term=&utm_campaign=&page={i}')
  return list_url_page

In [3]:
list_url_page = get_url_page_Jobsgo(200)

In [4]:
# Viết hàm lấy url job của các công việc
def get_url_job_Jobsgo(url_page):
  # Khởi tạo danh sách url các công việc
  list_url_job = []
  # Lấy nội dung của page
  Content = requests.get(url_page).content
  Soup = BeautifulSoup(Content, 'html.parser')
  # Lấy url job
  url_jobs = Soup.select('body > section.section.wrap-1 > div > div > div > div > div > div > div.col-sm-9 > div.row-ajax > div.clearfix.colorgb-carousel-bk.v2 > div.item-bk.mrg-top-10 > div > div > div > article > div.brows-job-list > div > div > div.brows-job-position > h3 > a')
  for job in url_jobs:
    list_url_job.append(job['href'])
  
  return list_url_job

In [ ]:
list_url_job_Jobsgo = []
for url_page in tqdm(list_url_page):
  try:
    list_url_job_Jobsgo.extend(get_url_job_Jobsgo(url_page))
  except:
    print(f'{url_page} -> Error')

100%|██████████| 200/200 [00:00<00:00, 117983.23it/s]

https://jobsgo.vn/viec-lam.html?utm_source=job_list&utm_medium=&utm_term=&utm_campaign=&page=1 -> Error
https://jobsgo.vn/viec-lam.html?utm_source=job_list&utm_medium=&utm_term=&utm_campaign=&page=2 -> Error
https://jobsgo.vn/viec-lam.html?utm_source=job_list&utm_medium=&utm_term=&utm_campaign=&page=3 -> Error
https://jobsgo.vn/viec-lam.html?utm_source=job_list&utm_medium=&utm_term=&utm_campaign=&page=4 -> Error
https://jobsgo.vn/viec-lam.html?utm_source=job_list&utm_medium=&utm_term=&utm_campaign=&page=5 -> Error
https://jobsgo.vn/viec-lam.html?utm_source=job_list&utm_medium=&utm_term=&utm_campaign=&page=6 -> Error
https://jobsgo.vn/viec-lam.html?utm_source=job_list&utm_medium=&utm_term=&utm_campaign=&page=7 -> Error
https://jobsgo.vn/viec-lam.html?utm_source=job_list&utm_medium=&utm_term=&utm_campaign=&page=8 -> Error
https://jobsgo.vn/viec-lam.html?utm_source=job_list&utm_medium=&utm_term=&utm_campaign=&page=9 -> Error
https://jobsgo.vn/viec-lam.html?utm_source=job_list&utm_medium=&

In [150]:
len(list_url_job_Jobsgo)

9944

In [ ]:
list_url_job_Jobsgo

['https://jobsgo.vn/viec-lam/nhan-vien-tu-van-tra-gop-ha-noi-dong-nai-binh-duong-tp-ho-chi-minh-quang-ngai-tien-giang-19293803624.html?utm_source=job_list',
 'https://jobsgo.vn/viec-lam/nhan-vien-telesales-cho-vay-tien-mat-19330222502.html?utm_source=job_list',
 'https://jobsgo.vn/viec-lam/nhan-vien-kinh-doanh-sales-cham-soc-khach-hang-luong-cung-upto-12-trieu-thuong-19310247793.html?utm_source=job_list',
 'https://jobsgo.vn/viec-lam/ke-toan-tong-hop-di-lam-ngay-chi-chap-nhan-ung-vien-co-kinh-nghiem-nganh-du-lich-19310370004.html?utm_source=job_list',
 'https://jobsgo.vn/viec-lam/ke-toan-ban-hang-19354461017.html?utm_source=job_list',
 'https://jobsgo.vn/viec-lam/chuyen-vien-cham-soc-khach-hang-19245924070.html?utm_source=job_list',
 'https://jobsgo.vn/viec-lam/hcm-chuyen-vien-telesale-thu-nhap-tu-10-20tr-nghi-t7-cn-19336550316.html?utm_source=job_list',
 'https://jobsgo.vn/viec-lam/hcm-nhan-vien-kinh-doanh-thi-truong-trinh-duoc-vien-thu-nhap-len-den-25-trieu-19139777027.html?utm_sourc

In [152]:
df_url_job_Jobsgo = pd.DataFrame({'URL Job Jobsgo': list_url_job_Jobsgo})

In [153]:
df_url_job_Jobsgo.to_csv(PATH+'/job_url_Jobsgo.csv',index_label='ID')

In [3]:
data = pd.read_csv(PATH+'/job_url_Jobsgo.csv')

In [ ]:
def Crawl_job_data(url):
    content_job = requests.get(url).content
    soup_job = BeautifulSoup(content_job, 'html.parser')

    # Tiêu đề công việc
    job_title = soup_job.find('h1', {'class': 'media-heading text-semibold'}).text.strip()
    job_title = re.sub(r"\[.*?\]", "", job_title).strip()

    # Tên công ty
    try:
        company_name = soup_job.select('body > section.section.colorgb-single.wrap-1.padd-top-0 > div > div > div > div > div.col-sm-4.job-detail-col-2 > div > div.panel.mb-15.panel-flat > div > div.profile-cover > div.media > div.media-body > h2 > a')[0].text.strip()
    except:
        company_name = soup_job.select('body > section.section.colorgb-single.wrap-1.padd-top-0 > div > div > div > div > div.col-sm-4.job-detail-col-2 > div > div.panel.mb-15.panel-flat.hidden-xs > div > div.media > div.media-body > h2 > a')[0].text.strip()

    # Quy mô công ty
    company_info = soup_job.select('body > section.section.colorgb-single.wrap-1.padd-top-0 > div > div > div > div > div.col-sm-4.job-detail-col-2 > div > div.panel.mb-15.panel-flat > div > div.company-info.text-grey > div.padd-20.padd-t-0 > div > span')
    if company_info:
        pass
    else:
        company_info = soup_job.select('body > section.section.colorgb-single.wrap-1.padd-top-0 > div > div > div > div > div.col-sm-4.job-detail-col-2 > div > div.panel.mb-15.panel-flat.hidden-xs > div > div.company-info.text-grey > div.padd-20.padd-0 > div > span')
    Industry = company_info[0].text.strip()
    if len(company_info) == 2:
        company_address = company_info[1].text.strip()
    else:
        company_address = company_info[2].text.strip()


    # Mô tả công việc
    description = soup_job.select('div.content-group > div.clearfix')[0]
    job_description = '. '.join(
            p.get_text(strip=True).lstrip("•").strip() for p in description if p.get_text(strip=True))

    job_description = re.sub(r'\.+', '.', job_description)
    job_description = re.sub(r"-\s*", "", job_description)
        

    # Yêu cầu công việc
    requirements = soup_job.select('div.content-group > div.clearfix')[1]
    job_requirement = '. '.join(
            p.get_text(strip=True).lstrip("•").strip() for p in requirements if p.get_text(strip=True))
    job_requirement = re.sub(r'\.+', '.', job_requirement)
    job_requirement = re.sub(r"-\s*", "", job_requirement)

    # Phúc lợ
    benefits = soup_job.select('div.content-group > div.clearfix')[2]
    job_welfares = '. '.join(
            p.get_text(strip=True).lstrip("•").strip() for p in benefits if p.get_text(strip=True))
    job_welfares = re.sub(r'\.+', '.', job_welfares)
    job_welfares: str = re.sub(r"-\s*", "", job_welfares)

    # Mức lương
    salary = soup_job.find('span', {'class': 'saraly'}).text.strip()
    
    # Ngày hết hạn
    expire_date = soup_job.select('body > section.section.colorgb-single.wrap-1.padd-top-0 > div > div > div > div > div.col-sm-8.pr0.job-detail-col-1 > div > div > div.media.stack-media-on-mobile.text-left.content-group.pb-0.mrg-top-10 > div.media-body-2 > ul > li:nth-child(1) > span')
    expire_date = ' '.join(
        p.get_text(strip=True).lstrip("•").strip() for p in expire_date if p.get_text(strip=True))

    elements = soup_job.find_all('p', {'class': 'sc-df6f4dcb-0 ioTZSy'})
    employee_rank = soup_job.select('body > section.section.colorgb-single.wrap-1.padd-top-0 > div > div > div > div > div.col-sm-8.pr0.job-detail-col-1 > div > div > div:nth-child(5) > div:nth-child(2) > div > p:nth-child(3)')[0].text.strip()  # Cấp bậc
    job_skill = soup_job.select('body > section.section.colorgb-single.wrap-1.padd-top-0 > div > div > div > div > div.col-sm-8.pr0.job-detail-col-1 > div > div > div:nth-child(6) > div:nth-child(2) > div > div > a')
    job_skill = ', '.join(
            p.get_text(strip=True).lstrip("•").strip() for p in job_skill if p.get_text(strip=True)) # Kĩ năng
    job_field = soup_job.select('body > section.section.colorgb-single.wrap-1.padd-top-0 > div > div > div > div > div.col-sm-8.pr0.job-detail-col-1 > div > div > div:nth-child(6) > div:nth-child(1) > div > div > a')
    job_field = ', '.join(
            p.get_text(strip=True).lstrip("•").strip() for p in job_field if p.get_text(strip=True)) # Lĩnh vực
    experiences = soup_job.select('body > section.section.colorgb-single.wrap-1.padd-top-0 > div > div > div > div > div.col-sm-8.pr0.job-detail-col-1 > div > div > div:nth-child(5) > div:nth-child(4) > div > p:nth-child(3)')[0].text.strip() # Số năm kinh nghiệm
    job_address = soup_job.select('body > section.section.colorgb-single.wrap-1.padd-top-0 > div > div > div > div > div.col-sm-8.pr0.job-detail-col-1 > div > div > div:nth-child(7) > div > div.div > div > div > p:nth-child(1)')[0].text.strip() # Địa chỉ làm việc
    job_address = re.split(r"\n", job_address)[0].strip()
    job_address = re.sub(r"-\s*", "", job_address)

    return [job_title, company_name, Industry, company_address, job_description, job_requirement, job_welfares, salary, expire_date, employee_rank, job_skill, job_field, experiences, job_address]

In [153]:
job_url, job_title, company_name, Industry, company_address, job_description, job_requirement = [], [], [], [], [], [], []
job_welfares, salary, expire_date, employee_rank, job_skill, job_field, experiences, job_address = [], [], [], [], [], [], [], []

In [154]:
for url in tqdm(data["URL Job Jobsgo"]):
  try:
    job_data_TCV = Crawl_job_data(url)
    job_url.append(url)
    job_title.append(job_data_TCV[0])
    company_name.append(job_data_TCV[1])
    Industry.append(job_data_TCV[2])
    company_address.append(job_data_TCV[3])
    job_description.append(job_data_TCV[4])
    job_requirement.append(job_data_TCV[5])
    job_welfares.append(job_data_TCV[6])
    salary.append(job_data_TCV[7])
    expire_date.append(job_data_TCV[8])
    employee_rank.append(job_data_TCV[9])
    job_skill.append(job_data_TCV[10])
    job_field.append(job_data_TCV[11])
    experiences.append(job_data_TCV[12])
    job_address.append(job_data_TCV[13])
  except:
    print('Error')

  0%|          | 0/9944 [00:00<?, ?it/s]

  0%|          | 26/9944 [00:08<52:48,  3.13it/s] 

Error


  1%|          | 56/9944 [00:17<51:40,  3.19it/s]

Error


...

100%|█████████▉| 9932/9944 [59:21<00:03,  3.62it/s]

Error


100%|█████████▉| 9943/9944 [59:24<00:00,  3.61it/s]

Error


100%|██████████| 9944/9944 [59:25<00:00,  2.79it/s]


In [155]:
Dataset_Jobsgo = pd.DataFrame({'URL Job': job_url,
                            'Job Title': job_title,
                            'Company Name': company_name,
                            'Industry': Industry,
                            'Company Address': company_address,
                            'Job Description': job_description,
                            'Job Requirements': job_requirement,
                            'Benefits': job_welfares,
                            'Salary': salary,
                            'Expire date': expire_date,
                            'Employee Rank': employee_rank,
                            'Job Skill': job_skill,
                            'Job Field': job_field,
                            'Years of Experience': experiences,
                            'Job Address': job_address})



In [ ]:
Dataset_Jobsgo.to_csv(PATH+'/Dataset_Jobsgo', index = False)

In [157]:
Dataset_Jobsgo

,URL Job,Job Title,Company Name,Industry,Company Address,Job Description,Job Requirements,Benefits,Salary,Expire date,Employee Rank,Job Skill,Job Field,Years of Experience,Job Address
0,https://jobsgo.vn/viec-lam/nhan-vien-tu-van-tr...,"Nhân Viên Tư Vấn Trả Góp - Hà Nội, Đồng Nai, B...",Công ty Tài Chính TNHH MTV Home Credit Việt Nam,Tài chính,"20 Nguyễn Đăng Giai, P.Thảo Điền, Tp.Thủ Đức, ...",Tìm kiếm khách hàng thông qua data công công t...,"Nam/Nữ, từ 18 – 33 tuổi. Tốt nghiệp THPT trở l...",Thu nhập từ8.000.000 15.000.000đồng/tháng (hưở...,8 - 15 triệu VNĐ,Hết hạn trong 20 ngày nữa,Nhân Viên/Chuyên Viên,"Chăm Sóc Khách Hàng, Tư Vấn Bán Hàng, Bán Lẻ","Chăm Sóc Khách Hàng, Kinh Doanh/Bán Hàng, Bán ...",Không yêu cầu,Đồng Nai
1,https://jobsgo.vn/viec-lam/nhan-vien-telesales...,Nhân Viên Telesales (Cho Vay Tiền Mặt),Công Ty Tài chính TNHH HD SAISON,Kinh doanh,"Lầu 8,9,10 Tòa nhà Gilimex, số 24C đường Phan ...",Đại diện HD SAISON trao đổi với khách hàng tro...,1. Trình độ học vấn:Tốt nghiệp Trung học phổ t...,Thử việc 100% lương;Ký hợp đồng lao động chính...,8 - 15 triệu VNĐ,Hết hạn trong 25 ngày nữa,Nhân Viên/Chuyên Viên,"Tư Vấn Bán Hàng, Bán Hàng Qua Điện Thoại (Tele...","Tài Chính/Ngân Hàng, Kinh Doanh/Bán Hàng, Bán ...",Không yêu cầu,"Tầng 8, 24C Phan Đăng Lưu, P6, Bình Thạnh, HCM"
2,https://jobsgo.vn/viec-lam/nhan-vien-kinh-doan...,Nhân Viên Kinh Doanh/ Sales/ Chăm Sóc Khách Hà...,Công Ty TNHH Tư Vấn & Đào Tạo Đại Dương,Giáo dục/ Đào tạo/ Tin học,"46A Đường 385, Phường Tăng Nhơn Phú A, Thành p...","1. Tìm kiếm, xây dựng mối quan hệ và phát triể...",Tốt nghiệp Cao Đẳng trở lên.Tuổi không quá 30....,Lương cứng từ9.000.000 – 12.000.000VNĐ/tháng+ ...,9 - 12 triệu VNĐ,Hết hạn trong 22 ngày nữa,Nhân Viên/Chuyên Viên,"Chăm Sóc Khách Hàng, Tư Vấn Bán Hàng","Tài Chính/Ngân Hàng, Giáo Dục/Đào Tạo, Kinh Do...",Trên 1 năm,"25 Đường Số 23, Linh Chiểu, Thủ Đức, Hồ Chí Minh"
3,https://jobsgo.vn/viec-lam/ke-toan-tong-hop-di...,Kế Toán Tổng Hợp - Đi Làm Ngay (Chỉ Chấp Nhận ...,Công Ty TNHH TravelCraft,Kinh doanh,"L18-11-13, Tầng 18, Tòa nhà Vincom Center Đồng...",Phụ trách thực hiện các công việc kế toán của ...,"Nam/Nữ, 25 40 tuổi;Tốt nghiệp Đại học chuyên n...",Lương: 25 Tr 30 Tr (Thỏa thuận theo năng lực ứ...,25 - 30 triệu VNĐ,Hết hạn trong 22 ngày nữa,Nhân Viên/Chuyên Viên,"Kế Toán Tổng Hợp, Kế Toán Quản Trị, Misa","Tài Chính/Ngân Hàng, Du Lịch, Kế Toán/Kiểm Toán",Trên 5 năm,"64 Bà Huyện Thanh Quan, Võ Thị Sáu, Quận 3, Hồ..."
4,https://jobsgo.vn/viec-lam/ke-toan-ban-hang-19...,Kế Toán Bán Hàng,Công Ty TNHH Thương Mại Và Xuất Nhập Khẩu Bình...,Sản xuất/Vận hành sản xuất,"Số 248, đường Nguyễn Văn Linh, tổ 17, Phường T...",Thực hiện các nghiệp vụ kế toán liên quan đến ...,Nữ tuổi từ 22 – 33KHÔNG CÓ KINH NGHIỆM CÓ THỂ ...,Thu Nhập: 8 10 triệu ( có thể deal theo năng l...,8 - 10 triệu VNĐ,Hết hạn trong 28 ngày nữa,Nhân Viên/Chuyên Viên,"Kế Toán Tổng Hợp, Kế Toán Bán Hàng, Tư Vấn Bán...","Kế Toán/Kiểm Toán, Kinh Doanh/Bán Hàng, Bán Sỉ...",Không yêu cầu,"244 Nguyễn Văn Linh, Cổ Bi, Gia Lâm, Hà Nội"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9511,https://jobsgo.vn/viec-lam/nhan-vien-sale-1920...,Nhân Viên Sale,Công Ty TNHH Nhôm Tiến Đạt,Sản xuất/Vận hành sản xuất,"135A/3 KP.Bình Đường 3, Bình An, Dĩ An, Bình D...",Đi khắp 13 tỉnh ( có hỗ trợ xe hơi đi) tiếp th...,Liên hệ [protected info]( a duc) để xếp lịch p...,"Môi trường làm việc dân chủ, hiện đại, chuyên ...",10 - 12 triệu VNĐ,Hết hạn trong 8 ngày nữa,Nhân viên/Chuyên viên,"Tư Vấn Bán Hàng, Bán Hàng Qua Điện Thoại (Tele...","Chăm Sóc Khách Hàng, Kinh Doanh/Bán Hàng",Không yêu cầu,Hậu Giang
9512,https://jobsgo.vn/viec-lam/fresher-mobile-flut...,Fresher Mobile (Flutter),Công Ty Cổ Phần Voyager,Công nghệ thông tin,"Floor 07, HOSE Building, Street 03, Quang Trun...","Tham gia học hỏi, xây dựng, phát triển các ứng...",Tốt nghiệp Đại học hoặc Cao đẳng trở lên các c...,Sẽ trở thành thành viên của một Đại Gia Đình đ...,Thỏa thuận,Hết hạn trong 8 ngày nữa,Mới tốt nghiệp/Thực tậ

In [9]:
data_VNW = pd.read_csv(PATH + "Dataset_VNW")
data_VNW

,URL Job,Job Title,Company Name,Industry,Company Address,Job Description,Job Requirements,Benefits,Salary,Expire date,Employee Rank,Job Skill,Job Field,Years of Experience,Job Address
0,https://www.vietnamworks.com/senior-expert-eco...,"Senior Expert, Economics and Financial Markets...",Techcombank,Ngân hàng,"Head Office: 6 Quang Trung Street, Hoan Kiem D...",Job Purpose. The job holder responsible for:. ...,"Degree: University degree in Finance, Economic...",13th month salary + performance bonus. Advance...,"$ 1,000-2,500 /tháng",Hết hạn trong 6 ngày,Trưởng phòng,"Banking, Financial Analysis, Macro Analysis, F...",Ngân Hàng & Dịch Vụ Tài Chính > Phân Tích & Bá...,10,"Số 6 Quang Trung, Hoàn Kiếm, HN"
1,https://www.vietnamworks.com/audit-senior-ha-n...,Audit Senior ( Ha Noi & HCM),"Crowe Vietnam Co., Ltd",Kế toán/Kiểm toán,"HAN Office: Room 1205, 12th Floor, IPH Tower, ...",1. The primary responsibility is to assist aud...,University degree in Auditing/Accounting/ Fina...,Work for an international brand that frequentl...,Thương lượng,Hết hạn trong 2 ngày,Nhân viên,"Audit, Analytical Skills, Kiểm Toán Nội Bộ, En...",Kế Toán/Kiểm Toán > Kiểm Toán,3,"Hồ Chí Minh, Việt Nam"
2,https://www.vietnamworks.com/qa-engineer-ky-su...,"QA Engineer (Kỹ Sư Chất Lượng, Tiếng Anh Thành...",Công Ty TNHH LG Innotek Việt Nam Hải Phòng,Điện/Điện tử,"Lô I, 4, Khu công nghiệp Tràng Duệ, Xã Hồng Ph...",Process Quality Assurance. The Process Quality...,Well basis quality concept. English communicat...,Công ty cung cấp ký túc xá (nếu cần thiết). Xe...,"Tới $ 1,000 /tháng",Hết hạn trong 19 ngày,Nhân viên,"Process Quality Assurance, Excel, Test Plannin...",Sản Xuất > Đảm Bảo Chất Lượng/Kiểm Soát Chất L...,1,"Hai Phong, Vietnam"
3,https://www.vietnamworks.com/truong-nhom-kinh-...,Trưởng Nhóm Kinh Doanh,AIA Exchange HCM,Bảo hiểm,"Vincom Center, 45 Lý Tự Trọng, Phường Bến Nghé...",Lên kế hoạch tuyển dụng và phát triển đội ngũ ...,Độ tuổi: 27 40. Có kinh nghiệm ít nhất 1 năm l...,Thu nhập: Ít nhất từ 25 40 triệu mỗi tháng. Tr...,25tr-40tr ₫/tháng,Hết hạn trong 27 ngày,Nhân viên,"Quản Trị Đội Ngũ, Kỹ năng bán hàng, Quản Lý Tà...",Kinh Doanh > Bán Hàng/Phát Triển Kinh Doanh,1,"Tầng 15, Vincom Đồng Khởi, 45A Lý Tự Trọng, Q1..."
4,https://www.vietnamworks.com/truong-phong-kinh...,Trưởng Phòng Kinh Doanh Kênh Lẻ (Chi nhánh Miề...,Công Ty Cổ Phần Anh Khuê Watch,Bán lẻ/Bán sỉ,"20 đường 3/2 Phường 12, quận 10, TP.HCM",Xây dựng và triển khai kế hoạch kinh doanh địn...,Giới tính: Nam/ Nữ (1990 – 1980). Tốt nghiệp Đ...,Thưởng theo hiệu quả công việc. Đầy đủ các chế...,Thương lượng,Hết hạn trong 23 ngày,Trưởng phòng,"Quản Lý Kinh Doanh Chuỗi, Kinh Doanh, Lập Kế H...",Kinh Doanh > Bán Hàng/Phát Triển Kinh Doanh,5,"Số 35 phố Cự Lộc, Km 6 Nguyễn Trãi, Phường Thư..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6174,https://www.vietnamworks.com/pqa-manager-staff...,PQA Manager/ Staff,Navigos Search's Client,Điện/Điện tử,Hanoi,Làm việc với các bộ phận liên quan để tiến hàn...,1. Tốt nghiệp trường đại học chuyên ngành tươn...,"13th monthly salary, Performance Bonus. Health...","$ 1,100-1,800 /tháng",Hết hạn trong 4 ngày,Nhân viên,"Quality Assurance, Product Quality Assurance, ...",Sản Xuất > Đảm Bảo Chất Lượng/Kiểm Soát Chất L...,Không yêu cầu,Ha Nam
6175,https://www.vietnamworks.com/export-sales-exec...,Export Sales Executive,Navigos Search's Client,Nông nghiệp/Lâm nghiệp/Nuôi trồng thủy sản,Hanoi,"Navigos Search's client, FDI company in Bac Ni...","Male, 2530 years old. Have experience in expor...","13th monthly salary, Performance Bonus. Health...","$ 700-1,000 /tháng",Hết hạn trong 4 ngày,Nhân viên,"Export, Sales, English",Kinh Doanh > Bán Hàng/Phát Triển Kinh Doanh,Không yêu cầu,Thành phố Hà Nội
6176,https://www.vietnamworks.com/hot-job-demand-pl...,Hot Job! Demand Planner (SAP S/4 Hana),Navigos Search's Client,Hoá chất/Hoá sinh,Hanoi / HCMC,NaN,NaN,"13th monthly salary, Performance Bonus. Health...","$ 1,200-1,800 /tháng",Hết hạn trong 4 ngày,Nhân viên,

In [10]:
data_Jobsgo = pd.read_csv(PATH + "Dataset_Jobsgo")

In [11]:
data_Jobsgo

,URL Job,Job Title,Company Name,Industry,Company Address,Job Description,Job Requirements,Benefits,Salary,Expire date,Employee Rank,Job Skill,Job Field,Years of Experience,Job Address
0,https://jobsgo.vn/viec-lam/nhan-vien-tu-van-tr...,"Nhân Viên Tư Vấn Trả Góp - Hà Nội, Đồng Nai, B...",Công ty Tài Chính TNHH MTV Home Credit Việt Nam,Tài chính,"20 Nguyễn Đăng Giai, P.Thảo Điền, Tp.Thủ Đức, ...",Tìm kiếm khách hàng thông qua data công công t...,"Nam/Nữ, từ 18 – 33 tuổi. Tốt nghiệp THPT trở l...",Thu nhập từ8.000.000 15.000.000đồng/tháng (hưở...,8 - 15 triệu VNĐ,Hết hạn trong 20 ngày nữa,Nhân Viên/Chuyên Viên,"Chăm Sóc Khách Hàng, Tư Vấn Bán Hàng, Bán Lẻ","Chăm Sóc Khách Hàng, Kinh Doanh/Bán Hàng, Bán ...",Không yêu cầu,Đồng Nai
1,https://jobsgo.vn/viec-lam/nhan-vien-telesales...,Nhân Viên Telesales (Cho Vay Tiền Mặt),Công Ty Tài chính TNHH HD SAISON,Kinh doanh,"Lầu 8,9,10 Tòa nhà Gilimex, số 24C đường Phan ...",Đại diện HD SAISON trao đổi với khách hàng tro...,1. Trình độ học vấn:Tốt nghiệp Trung học phổ t...,Thử việc 100% lương;Ký hợp đồng lao động chính...,8 - 15 triệu VNĐ,Hết hạn trong 25 ngày nữa,Nhân Viên/Chuyên Viên,"Tư Vấn Bán Hàng, Bán Hàng Qua Điện Thoại (Tele...","Tài Chính/Ngân Hàng, Kinh Doanh/Bán Hàng, Bán ...",Không yêu cầu,"Tầng 8, 24C Phan Đăng Lưu, P6, Bình Thạnh, HCM"
2,https://jobsgo.vn/viec-lam/nhan-vien-kinh-doan...,Nhân Viên Kinh Doanh/ Sales/ Chăm Sóc Khách Hà...,Công Ty TNHH Tư Vấn & Đào Tạo Đại Dương,Giáo dục/ Đào tạo/ Tin học,"46A Đường 385, Phường Tăng Nhơn Phú A, Thành p...","1. Tìm kiếm, xây dựng mối quan hệ và phát triể...",Tốt nghiệp Cao Đẳng trở lên.Tuổi không quá 30....,Lương cứng từ9.000.000 – 12.000.000VNĐ/tháng+ ...,9 - 12 triệu VNĐ,Hết hạn trong 22 ngày nữa,Nhân Viên/Chuyên Viên,"Chăm Sóc Khách Hàng, Tư Vấn Bán Hàng","Tài Chính/Ngân Hàng, Giáo Dục/Đào Tạo, Kinh Do...",Trên 1 năm,"25 Đường Số 23, Linh Chiểu, Thủ Đức, Hồ Chí Minh"
3,https://jobsgo.vn/viec-lam/ke-toan-tong-hop-di...,Kế Toán Tổng Hợp - Đi Làm Ngay (Chỉ Chấp Nhận ...,Công Ty TNHH TravelCraft,Kinh doanh,"L18-11-13, Tầng 18, Tòa nhà Vincom Center Đồng...",Phụ trách thực hiện các công việc kế toán của ...,"Nam/Nữ, 25 40 tuổi;Tốt nghiệp Đại học chuyên n...",Lương: 25 Tr 30 Tr (Thỏa thuận theo năng lực ứ...,25 - 30 triệu VNĐ,Hết hạn trong 22 ngày nữa,Nhân Viên/Chuyên Viên,"Kế Toán Tổng Hợp, Kế Toán Quản Trị, Misa","Tài Chính/Ngân Hàng, Du Lịch, Kế Toán/Kiểm Toán",Trên 5 năm,"64 Bà Huyện Thanh Quan, Võ Thị Sáu, Quận 3, Hồ..."
4,https://jobsgo.vn/viec-lam/ke-toan-ban-hang-19...,Kế Toán Bán Hàng,Công Ty TNHH Thương Mại Và Xuất Nhập Khẩu Bình...,Sản xuất/Vận hành sản xuất,"Số 248, đường Nguyễn Văn Linh, tổ 17, Phường T...",Thực hiện các nghiệp vụ kế toán liên quan đến ...,Nữ tuổi từ 22 – 33KHÔNG CÓ KINH NGHIỆM CÓ THỂ ...,Thu Nhập: 8 10 triệu ( có thể deal theo năng l...,8 - 10 triệu VNĐ,Hết hạn trong 28 ngày nữa,Nhân Viên/Chuyên Viên,"Kế Toán Tổng Hợp, Kế Toán Bán Hàng, Tư Vấn Bán...","Kế Toán/Kiểm Toán, Kinh Doanh/Bán Hàng, Bán Sỉ...",Không yêu cầu,"244 Nguyễn Văn Linh, Cổ Bi, Gia Lâm, Hà Nội"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9511,https://jobsgo.vn/viec-lam/nhan-vien-sale-1920...,Nhân Viên Sale,Công Ty TNHH Nhôm Tiến Đạt,Sản xuất/Vận hành sản xuất,"135A/3 KP.Bình Đường 3, Bình An, Dĩ An, Bình D...",Đi khắp 13 tỉnh ( có hỗ trợ xe hơi đi) tiếp th...,Liên hệ [protected info]( a duc) để xếp lịch p...,"Môi trường làm việc dân chủ, hiện đại, chuyên ...",10 - 12 triệu VNĐ,Hết hạn trong 8 ngày nữa,Nhân viên/Chuyên viên,"Tư Vấn Bán Hàng, Bán Hàng Qua Điện Thoại (Tele...","Chăm Sóc Khách Hàng, Kinh Doanh/Bán Hàng",Không yêu cầu,Hậu Giang
9512,https://jobsgo.vn/viec-lam/fresher-mobile-flut...,Fresher Mobile (Flutter),Công Ty Cổ Phần Voyager,Công nghệ thông tin,"Floor 07, HOSE Building, Street 03, Quang Trun...","Tham gia học hỏi, xây dựng, phát triển các ứng...",Tốt nghiệp Đại học hoặc Cao đẳng trở lên các c...,Sẽ trở thành thành viên của một Đại Gia Đình đ...,Thỏa thuận,Hết hạn trong 8 ngày nữa,Mới tốt nghiệp/Thực tậ

In [16]:
data_Jobsgo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9516 entries, 0 to 9515
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   URL Job              9516 non-null   object
 1   Job Title            9516 non-null   object
 2   Company Name         9516 non-null   object
 3   Industry             9440 non-null   object
 4   Company Address      9516 non-null   object
 5   Job Description      9516 non-null   object
 6   Job Requirements     9516 non-null   object
 7   Benefits             9510 non-null   object
 8   Salary               9516 non-null   object
 9   Expire date          9516 non-null   object
 10  Employee Rank        9516 non-null   object
 11  Job Skill            9405 non-null   object
 12  Job Field            9447 non-null   object
 13  Years of Experience  9516 non-null   object
 14  Job Address          9516 non-null   object
dtypes: object(15)
memory usage: 1.1+ MB


# Crawl user data

In [3]:
# Viết hàm lấy url page chứa danh sách các công việc
def get_url_page_user(num_page):
  list_url_page = []
  for i in range(1,num_page+1):
    list_url_page.append('https://www.alojob.com/ung-vien?page='+str(i))
  return list_url_page

In [4]:
list_url_page = get_url_page_user(1000)

In [21]:
list_url_page

['https://www.alojob.com/ung-vien?page=1',
 'https://www.alojob.com/ung-vien?page=2',
 'https://www.alojob.com/ung-vien?page=3',
 'https://www.alojob.com/ung-vien?page=4',
 'https://www.alojob.com/ung-vien?page=5',
 'https://www.alojob.com/ung-vien?page=6',
 'https://www.alojob.com/ung-vien?page=7',
 'https://www.alojob.com/ung-vien?page=8',
 'https://www.alojob.com/ung-vien?page=9',
 'https://www.alojob.com/ung-vien?page=10',
 'https://www.alojob.com/ung-vien?page=11',
 'https://www.alojob.com/ung-vien?page=12',
 'https://www.alojob.com/ung-vien?page=13',
 'https://www.alojob.com/ung-vien?page=14',
 'https://www.alojob.com/ung-vien?page=15',
 'https://www.alojob.com/ung-vien?page=16',
 'https://www.alojob.com/ung-vien?page=17',
 'https://www.alojob.com/ung-vien?page=18',
 'https://www.alojob.com/ung-vien?page=19',
 'https://www.alojob.com/ung-vien?page=20',
 'https://www.alojob.com/ung-vien?page=21',
 'https://www.alojob.com/ung-vien?page=22',
 'https://www.alojob.com/ung-vien?page=23

In [30]:
response = requests.get("https://www.alojob.com/ung-vien?page=1")

In [31]:
soup = BeautifulSoup(response.content, "html.parser")

In [32]:
elements = soup.find_all("a", {"class": "title-item text-bold prevent-click"})
for element in elements:
    print(f"https://www.alojob.com{element['href']}")

https://www.alojob.com/ung-vien/nguyen-thi-ngoc-thuy--75169
https://www.alojob.com/ung-vien/nguyen-thi-lan-vi-75168
https://www.alojob.com/ung-vien/tran-quoc-lam-75166
https://www.alojob.com/ung-vien/nguyen-thi-xuyen--75167
https://www.alojob.com/ung-vien/nguyen-thi-mai-trang-75165
https://www.alojob.com/ung-vien/doan-ngoc-nhat-vy--75164
https://www.alojob.com/ung-vien/huynh-le-tri-trung-75163
https://www.alojob.com/ung-vien/nguyen-tien-loc-75162
https://www.alojob.com/ung-vien/cao-le-thuy-hang-75158
https://www.alojob.com/ung-vien/duong-thi-ngoc-thuy-75159
https://www.alojob.com/ung-vien/pham-minh-nhat-75161
https://www.alojob.com/ung-vien/nguyen-nhat-vy-75157
https://www.alojob.com/ung-vien/nguyen-le-si--75156
https://www.alojob.com/ung-vien/dang-tan-dat-75155
https://www.alojob.com/ung-vien/le-thi-anh-thu--75154
https://www.alojob.com/ung-vien/y-phieu-hmok-75152
https://www.alojob.com/ung-vien/le-hoang-hai-75153
https://www.alojob.com/ung-vien/le-thi-kim-phuc--75151
https://www.aloj

In [33]:
def get_url_user(url_page):
  # Khởi tạo danh sách url các công việc
  list_url_user = []
  # Lấy nội dung của page
  Content = requests.get(url_page).content
  Soup = BeautifulSoup(Content, 'html.parser')
  # Lấy url user
  elements = soup.find_all("a", {"class": "title-item text-bold prevent-click"})
  for element in elements:
    list_url_user.append(f"https://www.alojob.com{element['href']}")
  
  return list_url_user

In [17]:
list_url_user = []
for url_page in tqdm(list_url_page):
  try:
    list_url_user.extend(get_url_user(url_page))
  except:
    print(f'{url_page} -> Error')

len(list_url_user)

100%|██████████| 1000/1000 [04:55<00:00,  3.39it/s]


20000